# Convert npy files to DICOM

In [1]:
import sys 
import os
sys.path.append(os.path.relpath("../code/"))
from data import *
from model import *
from codify import *

Using TensorFlow backend.
/home/jiamin/miniconda/envs/tf_gpu_cuda10/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jiamin/miniconda/envs/tf_gpu_cuda10/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jiamin/miniconda/envs/tf_gpu_cuda10/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

Import `.npy` files

In [2]:
# Parameters
width = 256
height = 256
slices = 20
channels = 4
train_path = '../npy_data/train/'
test_path = '../npy_data/test/'

params = {'IMG_WIDTH' : width,
          'IMG_HEIGHT' : height,
          'NUM_SLICES' : slices,
          'IMG_CHANNELS' : channels,
          'TRAIN_PATH' : train_path,
          'TEST_PATH' : test_path}

(X_train, X_test, y_train, y_test) = get_data(**params)

100%|██████████| 8/8 [00:00<00:00, 48.30it/s]

Done!


In [10]:
import datetime

def get_input_data(ID,SLICE,PATH,IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS):
    X_test = np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint16)
    dir_names = glob(PATH+ID+'/*')
    dir_names = natural_sort(dir_names)
    #1st ff, 2nd r2*, 3rd water, 4th fat
    #water ch = 0, ix = 2
    #fat ch = 1, ix = 3
    #ff ch = 2, ix = 0
    #r2* ch = 3, ix = 1
    def zero():
        return 0
    def one():
        return 1
    def two():
        return 2
    def three():
        return 3
    switcher = {
            0: two,
            1: three,
            2: zero,
            3: one
        }
    def ch_to_ix(argument):
        # Get the function from switcher dictionary
        func = switcher.get(argument, "nothing")
        # Execute the function
        return func()
    for ch in range(IMG_CHANNELS):
        dir_name = dir_names[ch_to_ix(ch)]
        _, ch_name = os.path.split(dir_name)
        path = glob(PATH+ID+'/'+ch_name+'/'+ID+'S'+ch_name+'I'+SLICE+'.DCM')
        img = dicom.read_file(path[0])
        img = (img.pixel_array).astype(np.int16)
        img = np.array(img, dtype=np.int16)[np.newaxis,:,:]
        X_test[:,:,:,ch] = img
    
    _, water_ch_name = os.path.split(dir_names[2])
    save_string = ID+'S'+water_ch_name+'I_mask_pred_'+SLICE+'.DCM'
    print('Done!')
    
    return X_test, save_string

def bytes_to_int(bytes):
    result = 0
    for b in bytes:
        result = result*256 + int(b)
    return result

In [22]:
json_file = open('../model/model-3.json','r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('../model/model-3.h5')

model.compile(optimizer=Adam(lr = 1e-4), loss='binary_crossentropy', metrics=[dice_coef,'accuracy'])

test_ids = next(os.walk(test_path+'imgs/0/'))[2]
DATA_PATH = '../data/Vertebrae/'
save_path = '../dcm_data/'

for id_ in test_ids:
    id_suffix = (id_.split("_",1)[1]).split(".",1)[0]
    print(id_suffix)
    files = glob(DATA_PATH + id_suffix + '/mask*/*.DCM')
    files = natural_sort(files)
    if not os.path.exists(save_path+id_suffix):
        os.makedirs(save_path+id_suffix)
    for file in files:
        slice_num = (file.split("mask_",1)[1]).split(".DCM",1)[0]
        (X_test, save_string) = get_input_data(id_suffix, slice_num, DATA_PATH, width, height, channels)
        # Predict on each slice and save masks as DICOM using the same slice's mask's metadata
        preds_test = model.predict(X_test, verbose=1)
        #print(preds_test.shape)
        preds_test = preds_test.squeeze()
        # Threshold predictions
        preds_test_t = (preds_test > 0.5).astype(np.uint16)
        #print(preds_test_t.shape)
        #plt.imshow(preds_test_t, cmap='gray')
        #plt.show()
        ds = dicom.dcmread(file)
        #print(ds)
        """code_lines = code_file(file)
        exec(code_lines)
        # Set the transfer syntax
        ds.is_little_endian = True
        ds.is_implicit_VR = True"""
        # Set creation date/time
        dt = datetime.datetime.now()
        ds.ContentDate = dt.strftime('%Y%m%d')
        timeStr = dt.strftime('%H%M%S.%f')  # long format with micro seconds
        ds.ContentTime = timeStr
        # Change pixel data to predicted values
        ds.PixelData = preds_test_t.tobytes()
        print("Writing test file", save_string)
        filename = save_path + id_suffix + '/' + save_string
        ds.save_as(filename)
        print("File saved.")
        """print('Load file {} ...'.format(filename))
        ds = dicom.dcmread(filename)
        print(ds)"""
    
    

E3544
Done!
1/1 [==============================] - 1s 600ms/step
Writing test file E3544S1001I_mask_pred_1.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_2.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_3.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_4.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_5.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_6.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_7.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test file E3544S1001I_mask_pred_8.DCM
File saved.
Done!
1/1 [==============================] - 0s 4ms/step
Writing test fi